In [ ]:
from surprise import Reader

In [ ]:
from surprise import Dataset

In [ ]:
import streamlit as st
import pandas as pd
import datetime
from surprise import Reader
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import ipywidgets as widgets
from IPython.display import display

In [ ]:
def get_top_anime_rating(data_frame, num_anime=15):
    if data_frame is None:
        print("Error: DataFrame is None.")
        return None
    # Drop rows with missing values in the "score" column
    data_frame.dropna(subset=['score'], inplace=True)
    if data_frame.empty:
        print("Error: DataFrame is empty after dropping missing values.")
        return None
    # Sort the DataFrame by "score" in descending order
    sorted_df = data_frame.sort_values(by="score", ascending=False)
    # Select the columns "title" and "score", then drop duplicates based on the "title" column
    unique_titles_df = sorted_df[["title", "score","img_url"]].drop_duplicates("title")
    # Take the specified number of top anime titles
    top_anime_titles = unique_titles_df.head(num_anime)
    return top_anime_titles


In [ ]:
def get_top_anime_popularity(data_frame, num_anime=15):

    sorted_df = data_frame.sort_values(by="members", ascending=False)
    # Select the columns "title" and "score", then drop duplicates based on the "title" column
    unique_titles_df = sorted_df[["title", "members","img_url"]].drop_duplicates("title")
    # Take the specified number of top anime titles
    top_anime_titles = unique_titles_df.head(num_anime)
    return top_anime_titles


In [ ]:
import pandas as pd

def get_top_unique_genre(anime_df):
    # Convert the "genre" column to string
    anime_df['genre'] = anime_df['genre'].astype(str)
    # Initialize an empty set to store unique genres
    unique_genres = set()
    # Iterate over each row in the DataFrame
    for genres_str in anime_df['genre']:
        # Split the genre string by comma and add each genre to the set of unique genres
        genres_list = [genre.strip() for genre in genres_str.strip("[]").replace("'", "").split(",") if genre.strip()]
        unique_genres.update(genres_list)

    return sorted(list(unique_genres))  # Sort and convert set to a list for clear output

In [ ]:
    # Define a function to get top titles for a given genre
def get_top_titles_for_genre(anime_df, genre, n_anime=20):
    # Convert the "genre" column to string
    anime_df['genre'] = anime_df['genre'].astype(str)
    # Filter DataFrame to include only rows containing the chosen genre
    filtered_df = anime_df[anime_df['genre'].str.contains(genre)]
    # Drop duplicates from the DataFrame
    filtered_df = filtered_df.drop_duplicates(subset="title")
    # Sort by "score"
    sorted_df = filtered_df.sort_values(by="score", ascending=False)
    # Get the top titles from the sorted DataFrame
    top_titles = sorted_df.head(n_anime)[["title", "img_url"]]

    return top_titles


In [ ]:
def get_top_anime_by_year(anime_df, year, n_anime=10):
    # include only rows for the specified year
    filtered_df = anime_df[anime_df["start_year"] == year]
    #filtered_df = filtered_df[~filtered_df['Type'].isin(['movie', 'Movie'])]
    filtered_df = filtered_df.drop_duplicates(subset="title")
    # Sort by "Rating"
    sorted_df = filtered_df.sort_values(by="score", ascending=False)
    # return the specified number of top anime titles and image URLs as a DataFrame
    top_anime_titles = sorted_df[["title", "img_url"]].head(n_anime)

    return top_anime_titles

In [ ]:
# keep using anime23
def find_similar_byanime(anime_name, anime_df, top_n=10):
    # Filter the anime DataFrame to get the data for the given anime
    anime_data = anime_df[~anime_df['type'].isin(['movie', 'Movie'])]
    anime_data = anime_data[anime_data["title"] == anime_name]

    if anime_data.empty:
        return pd.DataFrame(columns=["title", "img_url"])  # Return an empty DataFrame

    # Create a user-anime matrix based on the provided anime data
    user_anime_matrix = pd.pivot_table(data=anime_df,
                                       values='my_score',
                                       index='user_id',
                                       columns='anime_id',
                                       fill_value=0)

    # Compute the pairwise cosine similarity between the given anime and all other anime
    anime_vector = user_anime_matrix.loc[:, anime_data["anime_id"].values].values
    similarities = cosine_similarity(anime_vector.T, anime_vector.T)

    # Get the indices of the top N most similar anime (excluding the given anime itself)
    similar_anime_indices = similarities.argsort()[0][-2:-top_n-2:-1]

    # Get the anime IDs of the top N most similar anime
    similar_anime_ids = user_anime_matrix.columns[similar_anime_indices]

    # Get the titles of the top N most similar anime using the anime_df
    similar_anime_names = [anime_df.loc[anime_id, "title"] for anime_id in similar_anime_ids]

    # Create a DataFrame with similar anime titles and their image URLs
    similar_anime_df = pd.DataFrame({
        "title": similar_anime_names,
        "img_url": ""  # You can add the image URLs here if available
    })

    return similar_anime_df

In [ ]:
def get_surprise_recommendations(anime_df, user_name, num):
    # Check if the user name exists in the DataFrame
    if user_name not in anime_df['username'].unique():
        return "User not found in the anime_df."

    # Load data into Surprise Dataset
    reader = Reader(rating_scale=(1, 10))
    data = Dataset.load_from_df(anime_df[['user_id', 'anime_id', 'my_score']], reader)

    # Choose algorithm (SVD in this example)
    algo = SVD()

    # Split data into train and test sets
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

    # Train the model
    algo.fit(trainset)

    # Generate predictions for test set
    predictions = algo.test(testset)

    # Evaluate the model (optional)
    accuracy.rmse(predictions)

    # Get user ID based on user name
    user_id = anime_df.loc[anime_df['username'] == user_name, 'user_id'].iloc[0]

    # Get top-N recommendations for the specified user
    user_predictions = algo.test([(user_id, anime_id, 0) for anime_id in anime_df['anime_id'].unique()])
    top_recommendations = []
    for pred in sorted(user_predictions, key=lambda x: x.est, reverse=True)[:num]:
        anime_id = pred.iid
        title = anime_df.loc[anime_df['anime_id'] == anime_id, 'title'].iloc[0]
        rating = round(pred.est, 1)
        top_recommendations.append((title, rating))

    return top_recommendations

In [ ]:
def get_top_similar_anime_by_synopsis(anime_df, anime_name, num):
    # Define a TF-IDF Vectorizer
    tfidf = TfidfVectorizer(stop_words='english')
    # Replace NaN values with an empty string
    anime_df["synopsis"] = anime_df["synopsis"].fillna("")
    # Construct the TF-IDF matrix
    tfidf_matrix = tfidf.fit_transform(anime_df["synopsis"])
    # Compute the cosine similarity matrix
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    # Get the index of the anime matching the title
    idx = anime_df[anime_df['title'] == anime_name].index
    if not idx.empty:
        idx = idx[0]
        # Get the pairwise similarity scores of all anime with that anime
        sim_scores = list(enumerate(cosine_sim[idx]))
        # Sort the anime based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        # Get the top similar anime indices
        sim_indices = [i[0] for i in sim_scores[1:num+1]]  # Exclude the first item (itself) and get top num similar
        # Return the top similar anime
        return anime_df.iloc[sim_indices][['title', 'img_url']]
    else:
        return pd.DataFrame(columns=["title", "img_url"])  # Return an empty DataFrame if anime name not found


In [ ]:
def get_top_anime_rating_popularity(data_frame, num_anime=15, num_top=500):

    # Filter the DataFrame to exclude rows where the "type" column is "movie" or "Movie"
    #data_frame['episodes'] = pd.to_numeric(data_frame['episodes'], errors='coerce')

    # Filter rows where the "episodes" column is greater than 1
    #data_frame = data_frame[data_frame['episodes'] > 1]

    # Sort the DataFrame by "score" in descending order and drop duplicates based on "title"
    top_anime_rating = data_frame.sort_values(by="score", ascending=False).drop_duplicates("title").head(num_top)

    # Sort the resulting DataFrame by "popularity" in descending order
    top_anime_rating_popularity = top_anime_rating.sort_values(by="members", ascending=False)

    # Select the columns "title", "popularity", and "score", then drop duplicates based on "title"
    top_anime_Rating_Popularity = top_anime_rating_popularity[["title", "members", "score", "img_url"]].drop_duplicates("title").head(num_anime)

    return top_anime_Rating_Popularity

In [ ]:
def login(user_df):
    mail = st.text_input('Email', key='login-email')
    password = st.text_input('Password', type='password', key='login-password')

    if st.button('Login'):

        # Check if the email and password match
        match_condition = ((user_df['mail'] == mail) & (user_df['password'] == password))


        if match_condition.any():
            st.success("Login successful!")
        else:
            st.error("Invalid email or password. Please try again.")

In [ ]:

def create_account(csv_file_path):
    # Load user data CSV into DataFrame
    try:
        user_df = pd.read_csv(csv_file_path, dtype={'password': str})
    except FileNotFoundError:
        # Handle case where the CSV file does not exist
        user_df = pd.DataFrame(columns=['user_id', 'user_name', 'age', 'country', 'genre', 'mail', 'password'])

    # Generate user ID
    if user_df.empty:
        user_id = 1  # Start from 1 if DataFrame is empty
    else:
        user_id = user_df['user_id'].max() + 1

    user_name = st.text_input('Username', key='create-username')
    age = st.number_input('Age', min_value=0, max_value=150, key='create-age')
    country = st.text_input('Country', key='create-country')
    genre = st.text_input('Genre', key='create-genre')
    mail = st.text_input('Email', key='create-email')
    password = st.text_input('Password', type='password', key='create-password')

    if st.button('Create Account'):
        new_user = pd.DataFrame({
            "user_id": [user_id],  # Assign the generated user ID
            "user_name": [user_name],
            "age": [age],
            "country": [country],
            "genre": [genre],
            "mail": [mail],
            "password": [password]
        })

        user_df = pd.concat([user_df, new_user], ignore_index=True)

        try:
            user_df.to_csv(csv_file_path, index=False)
            st.success("Account created successfully!")
        except Exception as e:
            st.error(f"Failed to save account information: {e}")

    # Add return button to go back to login page
    if st.button('Return to Login', key='create-return'):
        return True

    return False

In [ ]:
# Function to add anime to user's list
def add_to_user_list(user_name, title, img_url, rating):
    global user_list

    # Check if user_list is empty or does not exist
    if user_list.empty:
        # Create a new DataFrame with the new row
        user_list = pd.DataFrame({'user_name': [user_name], 'title': [title], 'img_url': [img_url], 'rating': [rating]})
    else:
        # Concatenate the new row to user_list
        new_row = pd.DataFrame({'user_name': [user_name], 'title': [title], 'img_url': [img_url], 'rating': [rating]})
        user_list = pd.concat([user_list, new_row], ignore_index=True)

    # Save user_list to CSV file
    user_list.to_csv('/content/drive/MyDrive/user_list.csv', index=False)

In [ ]:
# Function to display user's list
def display_user_list(user_name):
    global user_list
    user_list = pd.read_csv('/content/drive/MyDrive/user_list.csv')  # Reload user_list from CSV

    st.write(f"Viewing My List for {user_name}...")

    # Filter user_list for the specified user_name
    user_list_filtered = user_list[user_list['user_name'] == user_name]

    # Display titles and images
    if not user_list_filtered.empty:
        for index, row in user_list_filtered.iterrows():
            st.image(row['img_url'], caption=row['title'], width=150)
            st.write(f"**Title:** {row['title']}")
            st.write(f"**Rating:** {row['rating']}")
    else:
        st.write("Your list is empty.")

In [ ]:
# Function to display anime information and options
def display_info(selected_anime, anime_df, user_df,user_name):
    anime_info = anime_df[anime_df['title'] == selected_anime]
    title = anime_info['title'].iloc[0]
    genre = anime_info['genre'].iloc[0]
    episodes = anime_info['episodes'].iloc[0]
    score = anime_info['score'].iloc[0]
    synopsis = anime_info['synopsis'].iloc[0]
    img_url = anime_info['img_url'].iloc[0]  # Assuming 'img_url' column contains image URLs

    # Display anime information
    st.write(f"**Title:** {title}")
    st.write(f"**Genre:** {genre}")
    st.write(f"**Episodes:** {episodes}")
    st.write(f"**Score:** {score}")
    st.write(f"**Synopsis:** {synopsis}")

    # Display anime image
    st.image(img_url, caption=title, width=300)  # Adjust width as needed

    # Provide options for rating, adding to My List, and accessing My List
    rating = st.selectbox("Rate this anime (1-10)", list(range(1, 11)))
    add_to_list = st.button("Add to My List")
    view_my_list = st.button("View My List")

    # Process user actions
    if rating:
        st.write(f"You rated '{title}' as: {rating}/10")

    if add_to_list:
        st.write(f"'{title}' added to My List")

        # Add anime to user's list
        add_to_user_list(user_name, selected_anime, img_url, rating)  # Call function to add anime to user's list

    if view_my_list:
        # Display user's list
        display_user_list(user_name)

In [ ]:
def Netero_chatBot() :
    # Initialize OpenAI client with your API key
    client = OpenAI(api_key="sk-proj-twv0jNl4WAZBOXCCoP1gT3BlbkFJX38wSiu4TXLbtetHisqp")
    st.markdown(
    "<h1 style='text-align: center; color: #FF5733;'> The Anime Alchemist</h1>",
    unsafe_allow_html=True
      )

    # Set page title
    st.title("Let's Talk Anime !")

    # Create a layout with two columns
    col1, col2 = st.columns([4, 2])  # Adjust the ratio as needed

    # Chat history
    if "messages" not in st.session_state:
        st.session_state.messages = []

    # User input
    user_input = col1.text_input("You:", key="user_input")
    if user_input:
        # Add user input to chat history
        st.session_state.messages.append({"role": "user", "content": user_input})

        # Placeholder for assistant's response
        assistant_response_placeholder = col1.empty()

        # Prompt OpenAI with user input and retrieve response
        openai_prompt = f"Question: {user_input}\nContext: Anime Chatbot\n"
        openai_response = client.completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=openai_prompt,
            max_tokens=200,
            stream=False
        ).choices[0].text.strip()

        # Display assistant's response
        if openai_response:
            assistant_response_placeholder.markdown(f"**Netero:** <span style='color:red'>{openai_response}</span>", unsafe_allow_html=True)

    # Display image in the second column
    col2.image("/content/drive/MyDrive/animedataset/isaac-netero-png-hunter-x-hunter-netero-png.png", width=400)

    return